# Introduction

Predict Your Custom image with the help of this notebook


In [ ]:
import gradio as gr
import numpy as np
from torchvision import transforms
import torch
from helper import *
import sys
import csv
from PIL import Image
from monoscene.models.monoscene import MonoScene

# Loading Model

In [ ]:
model_path = "/monoscene/MonoScene/trained_models/monoscene_kitti.ckpt"

csv.field_size_limit(sys.maxsize)
torch.set_grad_enabled(False)


model = MonoScene.load_from_checkpoint(
        model_path,
        dataset="kitti",
        n_classes=20,
        feature = 64,
        project_scale = 2,
        full_scene_size = (256, 256, 32),
    )


model.cuda()
model.eval()

img_W, img_H = 1220, 370

# Inference Custom Image

In [ ]:
def predict(img):
    img = np.array(img, dtype=np.float32, copy=False) / 255.0

    normalize_rgb = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )
    img = normalize_rgb(img)
   
    batch = get_projections(img_W, img_H)
    batch["img"] = img
    
    for k in batch:
        batch[k] = batch[k].unsqueeze(0).cuda()

    pred = model(batch)
    y_pred = torch.softmax(pred["ssc_logit"], dim=1).detach().cpu().numpy()
    y_pred = np.argmax(y_pred, axis=1)
    fig = draw(y_pred.squeeze(), batch['fov_mask_1'].cpu())
    return fig
   
color_image_path = "./input_images/000295.jpg"
image = Image.open(color_image_path)
predict(image)